In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

np.random.seed(1)

#if opened as a kaggle notebook, the data should be loaded correctly
#otherwise change the code accordingly

print('Loading train and test data...')
#df=pd.read_csv('../input/kepler-labelled-time-series-data/exoTrain.csv')
df=pd.read_csv('exoTrain.csv')
#print(df1)
#df2=pd.read_csv('../input/kepler-labelled-time-series-data/exoTest.csv')
df2=pd.read_csv('exoTest.csv')
#print(df2)

Loading train and test data...


In [9]:
#print(train_data)
test_data=np.array(df2,dtype=np.float32)
#print(test_data)


Y=df.loc[:,df.columns=="LABEL"]
X=df.loc[:,df.columns!="LABEL"]
ytest=test_data[:,0]
Xtest=test_data[:,1:]
# print(ytrain,'\n',Xtrain)
# print(ytest,'\n',Xtest)

m=1 # A chosen exoplanet host star's index for plott
n=100 # A chosen non-exoplanet host star's index

#print('Shape of Xtrain:',np.shape(Xtrain),'\nShape of ytrain:',np.shape(ytrain))

Xtrain =X
Y =Y.to_numpy()

# plt.plot(X.iloc[m],'r')
# plt.title('Light intensity vs time (for an exoplanet star)')
# plt.xlabel('Time index')
# plt.ylabel('Light intensity')
# plt.show()

# Fourier tranform, de-noising, normalising and scaling of the lightcurves.                                       

# Currently commented out as I am feeding the LSTM-GAN the original lightcurve

In [ ]:
from scipy.fftpack import fft,ifft

print('Applying Fourier Transform...')

Xtrain=Xtrain.to_numpy()
plt.figure(5)
plt.plot(Xtrain[1])
#Xtrain=np.abs(fft(Xtrain,n=len(Xtrain[m]),axis=1))


print(Xtrain,Xtrain.shape)

Xtrain=Xtrain[:,:1+int((len(Xtrain[0])-1)/2)]



Xtest=Xtest[:,:1+int((len(Xtest[0])-1)/2)]
plt.figure(1)
save_train_m = Xtrain[m]
plt.plot(Xtrain[m],'r')
plt.title('After FFT (for an exoplanet star)')
plt.xlabel('Frequency')
plt.ylabel('Feature value')
plt.show()

#### Normalizing

from sklearn.preprocessing import normalize

print('Normalizing...')
#Xtrain=normalize(Xtrain)
Xtest=normalize(Xtest)
plt.figure(2)
plt.plot(Xtrain[m],'r')
plt.title('After FFT,Normalization (for an exoplanet star)')
plt.xlabel('Frequency')
plt.ylabel('Feature value')
plt.show()




#### Applying Gaussian Filter

from scipy import ndimage

print('Applying Gaussian filter...')
#Xtrain=ndimage.filters.gaussian_filter(Xtrain,sigma=10)
Xtest=ndimage.filters.gaussian_filter(Xtest,sigma=10)
plt.figure(3)
plt.plot(Xtrain[m],'r')
plt.title('After FFT,Normalization and Gaussian filtering (for an exoplanet star)')
plt.xlabel('Frequency')
plt.ylabel('Feature value')
plt.show()




#### Scaling down the data

from sklearn.preprocessing import MinMaxScaler

print('Applying MinMaxScaler...')
scaler=MinMaxScaler(feature_range=(0,1))
#Xtrain=scaler.fit_transform(Xtrain)
Xtest=scaler.fit_transform(Xtest)
plt.figure(4)
plt.plot(Xtrain[m],'r')
plt.title('After FFT,Normalization, Gaussian filtering and scaling (for an exoplanet star)')
plt.xlabel('Frequency')
plt.ylabel('Feature value')
plt.show()


# GAN MODEL. Run this cell to train the model and generate exo planet lightcurves

In [ ]:
#model deployment
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import LSTM,SimpleRNN
from keras.layers import Dropout,GRU,Flatten,Input
from keras.layers import TimeDistributed,Reshape
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from keras.optimizers import Adam,Adamax,Nadam
adam = Adam(learning_rate = 3E-2)
print(Xtrain.shape)
print(Y.shape)
#Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],Xtrain.shape[1],1))
print(Y)
#for i in range(len(Y)):
 #   if(Y[i]==2):
  #      Y[i] = -1
print(Y)


#the generator takes in 100 random points, its
# a fully connected NN and it outputs 1500 points which is about
#the length of a lightcurve
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=512,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=Xtrain.shape[1]))
    
    generator.add(Reshape((Xtrain.shape[1],1)))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam)
    return generator
g = create_generator()
g.summary()

#The discriminator takes in the output generated by the generator function
#  which produces the lightcurve and the discriminator then outputs 1 or -1 
# checking whether it is an exo-planet or not
def create_discriminator():
    model = Sequential()
    model.add(LSTM(units =256,input_shape=(Xtrain.shape[1],1),return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(units =256,return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(units =128,return_sequences = False))
    model.add(Dropout(0.3))
    model.add(Dense(units =1,activation = "tanh"))
    model.compile(loss='binary_crossentropy',optimizer="rmsprop",metrics = ["accuracy"])
    return model
d =create_discriminator()
d.summary()

#This merges the 2 models together to make the GAN
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)

    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='rmsprop')
    return gan

gan = create_gan(d,g)
gan.summary()



#This is how a GAN is supposed to be trained for handwritten digit recognition
#I have changed the code slightly so that the dimensions are correct for
# the LSTM block during training
def training(epochs=1, batch_size=128):
    
    #Loading the data
    X_train, y_train = Xtrain,Y
    batch_count = X_train.shape[0] / batch_size
    X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 100])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            X = np.reshape(X,(X.shape[0],X.shape[1],1))
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 5 == 0:
            print(generated_images.shape)
        
            for i in range(0,batch_size):
                plt.figure()
                plt.plot(generated_images[i])
            plt.show()
        if e==epochs:
            for i in range(0,batch_size,2):
                plt.figure()
                plt.plot(generated_images[i])
training(300,10)
    

# generic LSTM Model, with and without time distributed layer, this is a different architecture from a GAN

In [ ]:
#model deployment

# print(Xtrain.shape)
# Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],Xtrain.shape[1],1))
# print(Xtrain.shape)
# print(Xtrain)
X_tester = Xtrain
xtrain_t = []
ytrain_t = []
graph = []
for j in range(1):
    for i in range(1,len(X_tester[0])):
        graph.append(X_tester[j][i])
for j in range(1,3):
    for i in range(100,len(X_tester[0])):
        xtrain_t.append(X_tester[j][i-100:i])
        ytrain_t.append(X_tester[j][i])
xtrain_t = np.array(xtrain_t)
ytrain_t = np.array(ytrain_t)
print(xtrain_t.shape)
# print(xtrain_t)
xtrain_t = np.reshape(xtrain_t,(xtrain_t.shape[0],xtrain_t.shape[1],1))
# ytrain_t = np.reshape(ytrain_t,(ytrain_t.shape[0],1))
#print(xtrain_t)
print(xtrain_t.shape)
    

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,SimpleRNN
from keras.layers import Dropout,GRU,Flatten
from keras.layers import TimeDistributed
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, load_model
import tensorflow as tf
from keras.optimizers import Adam,Adamax,Nadam
adam = Adam(learning_rate = 3E-2)
model = Sequential()
model.add(LSTM(units =500,input_shape=(xtrain_t.shape[1],1),return_sequences = True))
model.add(Dense(units =1))
model.compile(loss='binary_crossentropy',optimizer="rmsprop",metrics = ["accuracy"])
model.summary()
model.fit(xtrain_t,ytrain_t,epochs =10,batch_size = 100)

In [ ]:

def timeDistributed():
    temp1 = model.predict(xtrain_t[:1])
    print(temp1.shape)
    print(xtrain_t[:1].shape)
   
   
    
    limit = 3
    trial =[]
    temp1 = xtrain_t[:1]
    for i in range(limit):
        temp1 = model.predict(temp1)
        temp1 =np.reshape(temp1,(temp1.shape[1]))
        for j in temp1:
            trial.append(j)
        temp1 = np.reshape(temp1,(1,temp1.shape[0],1))
    #plotting the results
    abs_trial = []
    for i in range(len(trial)):
        abs_trial.append(abs(trial[i]))
    plt.figure()  
    
    plt.plot(trial)
    plt.xlim(-30,1000)
    plt.title('Predicted lightcurve with FFT,Normalization, Gaussian filtering and scaling')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    
    #plotting the actual FFT lightcurve
    plt.figure()
    plt.plot(graph,color = "r")
    plt.xlim(-30,1000)
    plt.title('After FFT,Normalization, Gaussian filtering and scaling (for an exoplanet star)')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    #plt.plot(np.reshape(xtrain_t[:1],(xtrain_t[:1].shape[1])),color = "r")
    #plt.plot(np.reshape(temp1,(temp1.shape[1])),color = "b")
    plt.show()
    return trial
    
trial =timeDistributed()


def nonTimeDistributed():
    limit =1
    trial = []
    trial_t = xtrain_t[:1]

    for i in range(limit):
        yt = model.predict(trial_t)
        trial.append([yt[0][0]])
        trial_t = np.reshape(trial_t,(trial_t.shape[1]))
        trial_t =np.append(trial_t,yt[0][0])
        trial_t = trial_t[1:]
        trial_t = np.reshape(trial_t,(1,trial_t.shape[0],1))

    plt.plot(trial)


    xtester = model.predict(xtrain_t)
    print(trial)
    print(xtester)

    plt.figure(5)
    plt.plot(xtester[:1000],color ="r")
    plt.figure(6)
    plt.plot(ytrain_t[:1000],color = "b")
    plt.show()

In [ ]:

trial_ns = np.asarray(trial)
graph_ns = np.asarray(graph)
noise = np.random.normal(0,0.1,trial_ns.shape)

print(trial_ns.shape)
trial_ns = trial_ns + noise
noise = np.random.normal(0.02,0.09,graph_ns.shape)
graph_ns = graph_ns+noise
plt.figure()
plt.plot(trial_ns)
plt.figure()
plt.plot(graph_ns)
plt.figure()
plt.plot(save_train_m,color = "r")




In [ ]:
trial_ns_ifft = ifft(trial_ns)
plt.plot(trial_ns_ifft)
plt.xlim(100,1000)
plt.ylim(-0.015,0.015)